# Drought-related burn severity in the 2018 Woolsey Fire, southern California

**Author:** Adam Mansur<br>
**Mentors:** E. Natasha Stavros and Joseph McGlinchy<br>
**Course:** Earth Analytics Applications, CU Boulder, Summer 2021

In [ ]:
# Import required Python libraries needed for blog post. These include
# utility scripts used to share functions between notebooks (third group). 

import os
from pathlib import Path

import contextily as cx
import earthpy as et
import earthpy.plot as ep
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import rioxarray as rxr 
import seaborn as sns
import xarray as xr

from ea_drought_burn.config import CRS
from ea_drought_burn.utils import (
    aggregate,
    open_raster,
    plot_bands,
    plot_regression,
    reproject_match
)




# Set working directory to the earthpy data directory
os.chdir(os.path.join(et.io.HOME, "earth-analytics", "data", "woolsey-fire"))

# Scale default font size up
sns.set(font_scale=1.2, style="white")

# Set layout defaults
plt.rc("figure.constrained_layout", use=True, h_pad=15/72, w_pad=18/72)

In [ ]:
# Load Woolsey Fire boundary
woolsey_fire = gpd.read_file(os.path.join("shapefiles",
                                          "nifc_woolsey_perimeter",
                                          "2018-CAVNC-091023.shp")).to_crs(CRS)
crop_bound = woolsey_fire.geometry

# Load California boundary
ca_bound = gpd.read_file(os.path.join("shapefiles",
                                      "ca_state_boundary",
                                      "CA_State_TIGER2016.shp"))

# Load Santa Monica Mountain (SMM) data stack
smm_stack = open_raster(
    os.path.join("aviris-climate-vegetation", "SMMDroughtstack.dat"),
    crs=CRS,
    crop_bound=crop_bound
)

# Load and reproject MTBS dNBR
dnbr = open_raster(
    os.path.join("mtbs-burn-severity",
                 "ca3424011870020181108",
                 "ca3424011870020181108_20171215_20181215_dnbr.tif"),
    crs=CRS
)
dnbr.rio.write_nodata(np.nan, inplace=True)
dnbr = dnbr.where(dnbr != -9999, np.nan)
dnbr = reproject_match(dnbr, smm_stack[0])

# Load and reproject MTBS classified dNBR
dnbr_classified = open_raster(
    os.path.join("mtbs-burn-severity",
                 "ca3424011870020181108",
                 "ca3424011870020181108_20171215_20181215_dnbr6.tif"),
    crs=CRS
)
dnbr_classified.rio.write_nodata(np.nan, inplace=True)
dnbr_classified = dnbr_classified.where(dnbr_classified != -9999, np.nan)
dnbr_classified = reproject_match(dnbr_classified, smm_stack[0])

# Read mask used to aggregate data to the PRISM grid
prism_grid = open_raster(
    os.path.join("masks", "prism_grid.tif"),
    crs=CRS,
    crop_bound=crop_bound,
    masked=False
)


## Project Overview

Over two weeks in November 2018, the Woolsey Fire burned nearly 100,000 acres in the Santa Monica Mountains north of Malibu, California (Fig. 1). Three people were killed in the fire and over 1,600 structures were destroyed. Two other fires that ignited the same day as the Woolsey Fire caused widespread damage throughout California.

In [ ]:
# Create basemap of the Woolsey Fire scar
ax = woolsey_fire.to_crs(epsg=3857).plot(alpha=0.6, color="r", figsize=(8, 6))

dim = 0.4
axin = ax.inset_axes([-0.057, 1 - dim, dim, dim])
ca_bound.to_crs(epsg=3857).plot(ax=axin, facecolor="gainsboro", edgecolor="none")
woolsey_fire.envelope.to_crs(epsg=3857).plot(ax=axin, facecolor="k", edgecolor="k")

cx.add_basemap(ax, source=cx.providers.Stamen.TerrainBackground)

ax.set(title="Boundary of the Woolsey Fire", xticks=[], yticks=[])
axin.set(xticks=[], yticks=[])

pass

**Figure 1.** Woolsey Fire boundary. Inset shows the location of the study area in California.

Wildfires like the Woolsey Fire pose an increasing threat to life, property and ecosystems in the American West. Larger fires and longer fire seasons stress emergency response personnel and budgets while more people move into the urban-wildland interface where fires are more likely. Understanding the factors that drive wildfires may allow authorities better manage them, mitigating some risks in the fire-prone West.

Drought influences fire severity by drying out soil and killing vegetation that can serve as fuel. The Woolsey Fire was preceded by a four-year drought (2013-2016) that resulted in a significant dieback of grass, shrubs, and trees throughout California. How did that dieback influence the spread and severity of the Woolsey Fire? Previous work on insect-related diebacks does not offer a clear answer to this question, except to suggest that the effect of diebacks on fire severity depends strongly on local conditions (Bond et al., 2009; Hicke et al., 2012).

In this project, we will examine the relationship between vegetation mortality and burn severity during the Woolsey Fire by using a machine-learning approach to analyze pre- and post-fire satellite imagery. We plan to test the following hypotheses:

+ Areas with greater vegetation mortality will show worse burn severity
+ Areas with older vegetation mortality will show worse burn severity

## Data sources

This study leverages data collected for an earlier survey of the 2013-2016 drought (Foster et al., 2017; Table 1). That study's authors used high-resolution data from NASA's AVIRIS sensor to create vegetation community (Fig. 2) and fraction-alive (FAL) maps (Fig. 3) that document the health of the vegetation in the area later burned by the Woolsey Fire. They also compiled 4 km climate data to understand how different communities reacted to the prolonged drought. Using the FAL data, we also calculated fraction-alive difference (dFAL) plots to quantify changes in vegetation mortality during each year of the California drought.

We are also using burn severity data for the Woolsey Fire from Monitoring Trends in Burn Severity (MTBS), an interagency program that creates normalized, field-validated Normalized Burn Ratio difference (dNBR) plots for recent wildfires (Fig. 4).

### Table 1. Data Sources
| Data Type | Source | Description | Source Spatial Resolution |
| :---: | :--- | :---: | :---: |
| Vegetation Community | NASA | Classified vegetation communities;<br> derived from AVIRIS sensor data | 15.6 m | 
| Relative Fraction Alive (RFAL) | NASA | Proxy for healthy vegetation;<br> time series from 2013-16 | 15.6 m | 
| Difference Normalized Burned Ratio (dNBR) | MTBS | Indicator of burn severity; derived from <br> Sentinel-2 data from 2017 and 2018 | 30 m | 
| Digital Elevation Model (DEM) | USGS | Elevation data for characterizing terrain;<br>used to create derivatives such as slope and aspect | 1 m | 
| Weather | NRCS | Data layers obtained include Days of Precipitation, <br> Vapor Pressure Deficit, Minimum Temperature, <br> Days over 95 Degrees, Cumulative Precipitation. <br> These data represent annual aggregates for the years 2013-2016. | 4 km | 

In [ ]:
# Make vegetation community plot
labels = [
    "No data",
    "Annual grass",
    "Chaparral",
    "Coastal sage scrub",
    "Oak woodland",
    "Riparian",
    "Substrate"
]

colors = [
    "white",
    "tab:green",
    "tab:olive",
    "tab:cyan",
    "tab:red",
    "tab:blue",
    "lightgray"
]

# Draw plot
fig, ax = plt.subplots(figsize=(7, 7))
plot_bands(smm_stack[0],
           ax=ax,
           title="Vegetation community map - Woolsey Fire - 15.6 m",
           cmap=ListedColormap(colors),
           cbar=False)
woolsey_fire.plot(ax=ax, edgecolor="k", facecolor="none", linewidth=2)

# Draw legend
ep.draw_legend(ax.get_images()[0], 
               titles=labels,
               classes=range(len(labels)),
               bbox=(0.01, 0.99))

pass

**Figure 2.** Vegetation community map for the Woolsey Fire scar. Community data was provided by E. Natasha Stavros and was calculated using NASA AVIRIS data (15.6 m resolution).

In [ ]:
# Make fraction alive plot

# Pull FAL from the SMM stack
smm_fal = smm_stack[1:5]

# Setting non-positive values to the middle of the range allows matplotlib
# to interpolate the diverging color scheme more clearly
smm_fal = smm_fal.where(smm_fal >= 0, 0.5)

# Image has to be big or the interpolation is terrible
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))

plot_bands(smm_fal[0],ax=ax1, cmap="PiYG", vmin=0, vmax=1, title="FAL (2013)")
plot_bands(smm_fal[3], ax=ax2, cmap="PiYG", vmin=0, vmax=1, title="FAL (2016)")

for ax in (ax1, ax2):
    woolsey_fire.plot(ax=ax, edgecolor="k", facecolor="none", linewidth=2)
    
pass

**Figure 3.** Fraction-alive (FAL) rasters for 2013 (left) and 2016 (right). FAL is the fraction of green to total vegetation. FAL data was provided by E. Natasha Stavros and was calculated using NASA AVIRIS data (15.6 m resolution).

In [ ]:
# Make dNBR plot
dnbr_cat_names = [
    "Unburned to Low",
    "Low",
    "Moderate",
    "High",
    "Increased Greenness"
]

nbr_colors = [
    "#006400",
    "#7FFFD4",
    "#FFFF00",
    "#FF0000",
    "#7FFF00"
]
nbr_cmap = ListedColormap(nbr_colors)

fig, ax = plt.subplots(1, 1, figsize=(7, 7))
plot_bands(dnbr_classified.rio.clip(woolsey_fire.geometry),
           cmap=nbr_cmap,
           vmin=1,
           vmax=5,
           title=("MTBS classified dNBR - Woolsey Fire\n"
                  "Dec 15, 2017 - Dec 15, 2018"),
           cbar=False,
           scale=False,
           ax=ax)
woolsey_fire.plot(ax=ax, edgecolor="k", facecolor="none", linewidth=2)

ep.draw_legend(im_ax=ax.get_images()[0],
               classes=range(5),
               titles=dnbr_cat_names,
               bbox=(0.01, 0.99))

pass

**Figure 4.** Montoring Trends in Burn Severity (MTBS) classified dNBR plot. Calculated by MTBS based on Sentinel-2 (ESA) imagery (30 m resolution).

## Workflow

We plan to use the random-forest machine-learning algorithm to evaluate how vegetation mortality affects burn severity. As such, we must first select the explanatory and response variables we want to model.In order to characterize the relationships between the available data, we are analyzing pairwise linear regressions.

We plan to evaluate two aspects of the pre- and post-fire system, both of which have obvious response variables:

+ Drought-resistance of different vegetation communities (response=FAL or dFAL)
+ Burn severity (response=dNBR)

Selecting explanatory variables requires a closer look at the vegetation and climate data. Each data source (described in more detail below) provides data in a different coordinate reference system (CRS), resolution, and extent, so we projected all data to the same CRS, resampled it to the same resolution, and cropped it to the same extent before beginning our analysis. The reprojected data contains roughly 1.6 million pixels inside the Woolsey Fire scar. To reduce computation times, we've filtered the data using two approaches:

+ Creating training and validation datasets of a few thousand pixels a piece inside the fire scar. We will ultimately use these subsets to train and validate the random-forest model, but are currently using them to evaluate linear regressions.
+ Aggregating high resolution data into 4 km blocks.

## Preliminary results

The pixel data, even when reduced to a sample of a few thousand pixels, is very noisy. For analyses of drought resistance, a major source of noise is the low resolution of the climate data. Each 4 km block of climate data corresponds to thousands of points in the higher-resolution datasets, which span a range of vegetation types, slopes, and elevations, producing large spreads of data. But even the fire data is noisy at the scale of the full dataset or individual vegetation communities. We plan to address this problem as described below in the Future Work section.

The 4 km aggregate data has been more useful for finding correlations, but so far the only relationships that we've tested are between dNBR and FAL/dFAL. Burn severity correlates positively with (1) vegetation mortality in 2013 and (2) area of vegetation that died between 2013 and 2014. Fig. 5 shows preliminary correlations between dNBR and FAL for 4 km cells across the four years of the recent drought.


In [ ]:
# Increase the font sizes for the regression plots
sns.set(font_scale=1.1, style="white")

# Approximate upper limit of unburned is 100
burned = xr.where(dnbr >= 100, True, False).values

# Aggregate data and plot regressions
fig, axes = plt.subplots(1, 4, figsize=(16, 4.5))
fig.suptitle("Aggregated burn severity vs. fraction alive (2013-2016)")
axes = list(axes)

for i, year in enumerate(range(2013, 2017)):

    # Read x and y
    x = smm_fal[i].copy()
    y = dnbr.copy()

    # Mask nodata values
    x = x.where(x >= -1e38, np.nan)
    y = y.where(y >= -1e38, np.nan)
    
    # Mask unburned pixels
    x = x.where(burned)
    y = y.where(burned)

    # Aggregate along the PRISM grid
    x = aggregate(x, prism_grid)
    y = aggregate(y, prism_grid)
    
    # Convert to 1D arrays
    x = np.ravel(x)
    y = np.ravel(y)

    # Limit to values that are finite and nonzero in both arrays
    mask = x * y
    x = x[np.isfinite(mask)]
    y = y[np.isfinite(mask)]

    # Plot regression
    plot_regression(x, y, axes.pop(0),
                    color="gray",
                    title=year,
                    xlabel="FAL",
                    ylabel="dNBR")

**Figure 5.** Linear regressions between dNBR and FAL across all four years of the California drought. Data is aggregated to the 4 km grid used by the PRISM gridded climate data. FAL data was provided by E. Natasha Stavros and was calculated using NASA AVIRIS data. dNBR data was calculated by MTBS.

## Future work

We plan to address the noise in the sampled data by looking at specific subsets. So far, we have only looked at individual vegetation communities across the full study area, but we plan to combine community data with climate, elevation and slope data to identify subsets that respond clearly to drought or fire. We also plan to refine the aggregation methods mentioned above and investigate additional variables using the 4 km blocks.

Once we've selected the explanatory variables, we will begin running random-forest models to evaluate how well each explanatory value predicts a response.


## Acknowledgments

I would like to acknowledge Michael Sutherland for his contributions to this project as part of the Earth Analytics course in Spring 2021.

## References

+ Bond ML, Lee DE, Bradley CM, Hanson CT. 2008. Influence of Pre-Fire Tree Mortality on Fire Severity in Conifer Forests of the San Bernardino Mountains, California. Open For Sci Jour. 2: 41-47.


+ Foster K, Queally N, Nickmeyer A, Rousseau N. 2017. Utilizing NASA Earth Observations to Determine Drought Dieback and Insect-related Damage in the Santa Monica Mountains, California. Available from: https://develop.larc.nasa.gov/2017/fall/posters/2017Fall_JPL_SantaMonicaMountainsEcoII_Poster.pdf


+ Hicke JA, Johnson MC, Hayes JL, Presiler HK. 2012. Effects of bark beetle-caused tree mortality on wildfire. For Ecol Manag. 271: 81-90. DOI: [10.1016/j.foreco.2012.02.005](https://doi.org/10.1016/j.foreco.2012.02.005). 